In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from random import sample
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
import os
import random
from surprise.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
import gzip
import json

In [2]:
raw_data = pd.read_csv("Data/AMAZON_FASHION.csv", names=['Item', 'User', 'Rating', 'Timestamp'])
data = raw_data.sample(n=30000)
data.drop(['Timestamp'], axis=1,inplace=True)
data = data[['User', 'Item', 'Rating']]

In [3]:
data

,User,Item,Rating
99714,A1S51AWXWEFNNE,B009RUKQ2G,3.0
677538,A3UFBW6H6JUK7X,B0127UBXEC,5.0
6252,A2AG1J11CBBXGL,B000GHMRLW,5.0
493904,A3A63B9U421WT3,B00HVE5Q70,5.0
319034,A133MQ8GLM5RKO,B015MMA0GO,3.0
...,...,...,...
372619,A3UTT762EKOC5H,B01DPGKDPC,5.0
467667,A2TW5C1QXNJDQE,B00EKV2GFC,5.0
426317,A3IXGTCV960L0O,B0071NJDWA,4.0
134674,A36EHSHZY421JX,B00ECGFAIA,5.0


In [4]:
new_df=data.head(10000)
ratings_matrix = new_df.pivot_table(values='Rating', index='User', columns='Item', fill_value=0)
ratings_matrix.head()

Item,B0000AWHAA,B0000D9TB5,B0000ERU6T,B00015VKT4,B0001LU08U,B0002BKVZG,B0002Z1JNK,B00062NHH0,B00063VWSA,B00063W5J0,...,B01HH6E4IS,B01HHLJ90G,B01HHVC958,B01HI2584U,B01HI7K476,B01HIQH2W2,B01HIQZXBO,B01HIZ7Z4I,B01HJEOBN6,B01HJEOBUO
User,,,,,,,,,,,,,,,,,,,,,
A0054271FXCGYGCKW9TH,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A03175705Q7Z80DQ0AHA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0355357TBZ0Q1I0JVBH,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0813066D2W6LAAKNHC6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0952394J96P9CBWGK9Y,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X = ratings_matrix.T
X1 = X
X.head()

User,A0054271FXCGYGCKW9TH,A03175705Q7Z80DQ0AHA,A0355357TBZ0Q1I0JVBH,A0813066D2W6LAAKNHC6,A0952394J96P9CBWGK9Y,A1001I94O5V0O5,A1004U2SH7NX5E,A1005ZW1SZLUJP,A100BLX5CCQVFT,A100Q7SYKH1PLF,...,AZRQ1NVCROIYX,AZSHXSX0003NV,AZTSUW7JA3QT4,AZU666JF8WWWB,AZUOXQ1M4JYMG,AZVK67LM6PFQ1,AZVLBP2MJZRM8,AZWP047VIDYQH,AZXNDLIE4XNVT,AZXUR5N81IPIN
Item,,,,,,,,,,,,,,,,,,,,,
B0000AWHAA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B0000D9TB5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B0000ERU6T,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B00015VKT4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B0001LU08U,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
#Decomposing the Matrix

SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)
decomposed_matrix.shape

(7408, 10)

In [7]:
correlation_matrix = np.corrcoef(decomposed_matrix)
correlation_matrix.shape

(7408, 7408)

In [8]:
i = X.index[random.randint(0, np.size(correlation_matrix, 0))]

product_names = list(X.index)
product_ID = product_names.index(i)
product_ID

3937

In [9]:
correlation_product_ID = correlation_matrix[product_ID]
correlation_product_ID.shape

(7408,)

In [10]:
Recommend = list(X.index[correlation_product_ID > 0.65])

# Removes the item already bought by the customer
Recommend.remove(i) 

Recommend[0:24]

['B0002Z1JNK',
 'B00066G516',
 'B0007MV6PO',
 'B0007MV6Q8',
 'B00095LIES',
 'B0009F7SD8',
 'B0009NYJF0',
 'B0009QZAK0',
 'B000BCH8AC',
 'B000EB3RXM',
 'B000EM16IY',
 'B000EYBIJE',
 'B000EZIDOQ',
 'B000F4KPIS',
 'B000G01UBW',
 'B000HKLX4A',
 'B000I9OI6A',
 'B000IHVPLS',
 'B000IWK0GE',
 'B000J4J8BY',
 'B000J4KN18',
 'B000K0I840',
 'B000K2PJ4K',
 'B000OI56VG']

In [11]:
def parse(path):
  g = gzip.open(path, 'rt', encoding='utf-8')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

review_data = getDF('Data/meta_AMAZON_FASHION.json.gz')
data_set = review_data[:30000]
data_set
    

,title,image,brand,feature,rank,date,asin,description,price,also_view,also_buy,fit,details,similar_item,tech1
0,Slime Time Fall Fest [With CDROM and Collector...,[https://images-na.ssl-images-amazon.com/image...,Group Publishing (CO),[Product Dimensions:\n \n8....,"13,052,976inClothing,Shoesamp;Jewelry(",8.70 inches,0764443682,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,XCC Qi promise new spider snake preparing men'...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,"11,654,581inClothing,Shoesamp;Jewelry(",5 star,1291691480,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Magical Things I Really Do Do Too!,[https://images-na.ssl-images-amazon.com/image...,Christopher Manos,[Package Dimensions:\n \n8....,"19,308,073inClothing,ShoesJewelry(",5 star,1940280001,[For the professional or amateur magician. Ro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Ashes to Ashes, Oranges to Oranges",[https://images-na.ssl-images-amazon.com/image...,Flickerlamp Publishing,[Package Dimensions:\n \n8....,"19,734,184inClothing,ShoesJewelry(",5 star,1940735033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aether & Empire #1 - 2016 First Printing Comic...,[https://images-na.ssl-images-amazon.com/image...,NaN,[Package Dimensions:\n \n10...,"10,558,646inClothing,Shoesamp;Jewelry(",5 star,1940967805,NaN,$4.50,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29997,Honeystore Women's Cap Sleeve Crystal Evening ...,[https://images-na.ssl-images-amazon.com/image...,Honeystore,[Package Dimensions:\n \n15...,"22,005,842inClothing,ShoesJewelry(",5 star,B00F5V4JJ2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29998,Honeystore Women's Cap Sleeve Crystal Evening ...,[https://images-na.ssl-images-amazon.com/image...,Honeystore,[Package Dimensions:\n \n15...,"18,933,551inClothing,ShoesJewelry(",5 star5 star (0%),B00F5V4RDA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29999,AllCraft 1000x 14 Facets Resin Rhinestone Gem ...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,"5,687,062inClothing,ShoesJewelry(",jw00541,B00F5V9758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30000,Honeystore Women's Cap Sleeve Crystal Evening ...,[https://images-na.ssl-images-amazon.com/image...,Honeystore,[Package Dimensions:\n \n15...,"24,206,139inClothing,ShoesJewelry(",5 star,B00F5V4IAC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
top25 = pd.DataFrame({'ID': Recommend[0:25]})

In [13]:
top25

,ID
0,B0002Z1JNK
1,B00066G516
2,B0007MV6PO
3,B0007MV6Q8
4,B00095LIES
5,B0009F7SD8
6,B0009NYJF0
7,B0009QZAK0
8,B000BCH8AC
9,B000EB3RXM


In [14]:
top25.merge(data_set,left_on = 'ID', right_on = 'asin')

,ID,title,image,brand,feature,rank,date,asin,description,price,also_view,also_buy,fit,details,similar_item,tech1
0,B0002Z1JNK,Pleaser Women's Electra-2000Z Boot,[https://images-na.ssl-images-amazon.com/image...,Pleaser,"[100% Synthetic, Synthetic sole, Shaft measure...","261,979inClothing,Shoesamp;Jewelry(",5 star,B0002Z1JNK,[Shaft Height: Approx. 17'' Tall. Heel Height:...,$52.95 - $75.00,"[B0018NBXS6, B007UN3RP0, B0018NFT5E, B07BCLBQY...","[B07BMF139V, B0018NFT5E, B00Y8DE9ES, B001AH4NY...","class=""a-normal a-align-center a-spacing-smal...",NaN,NaN,NaN
1,B0002Z1JNK,Pleaser Women's Electra-2000Z Boot,[https://images-na.ssl-images-amazon.com/image...,Pleaser,"[100% Synthetic, Synthetic sole, Shaft measure...","261,979inClothing,Shoesamp;Jewelry(",5 star,B0002Z1JNK,[Shaft Height: Approx. 17'' Tall. Heel Height:...,$52.95 - $75.00,"[B0018NBXS6, B007UN3RP0, B0018NFT5E, B07BCLBQY...","[B07BMF139V, B0018NFT5E, B00Y8DE9ES, B001AH4NY...","class=""a-normal a-align-center a-spacing-smal...",NaN,NaN,NaN
2,B00066G516,Leg Avenue Women's Lace Ruffle Anklet Socks,[https://images-na.ssl-images-amazon.com/image...,NaN,"[100% Nylon Exclusive Of Decoration, Hand Wash...","32,865inClothing,ShoesJewelry(",5 star,B00066G516,NaN,$7.99,"[B074K1VNDZ, B01BLUW5HS, B01EXPYRCO, B016WXQAC...","[B07JKZR4BP, B01BLUW5HS, B01MQJLNLC, B016WXQAC...","class=""a-normal a-align-center a-spacing-smal...",NaN,NaN,NaN
3,B0007MV6PO,"Henschel Breezer Hat with CoolMax Band Khaki,M...",[https://images-na.ssl-images-amazon.com/image...,Henschel,[Package Dimensions:\n \n15...,"1,164,241inClothing,Shoesamp;Jewelry(",5 star,B0007MV6PO,NaN,NaN,[B00LNVG8ZG],"[B00LNVG8ZG, B00LNVG9PU, B01MSYE7YH, B001AF15H...",NaN,NaN,NaN,NaN
4,B0007MV6Q8,"Henschel Breezer Hat with CoolMax Band, Khaki, XL",[https://images-na.ssl-images-amazon.com/image...,Henschel,[Package Dimensions:\n \n15...,"1,045,705inClothing,Shoesamp;Jewelry(",5 star,B0007MV6Q8,NaN,NaN,[B00LNVG9PU],"[B00LNVG8ZG, B00KGGHTBQ, B00LNVG9PU, B01MSYE7Y...",NaN,NaN,NaN,NaN
5,B00095LIES,Barbie and The Magic of Pegasus: Magical Music...,[https://images-na.ssl-images-amazon.com/image...,Barbie,[Product Dimensions:\n \n20...,"13,866,976inClothing,ShoesJewelry(",5 star,B00095LIES,[Girls can experience the pretend magic of fli...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,B0009F7SD8,Small Southwestern Native American Handcrafted...,NaN,NaN,[Package Dimensions:\n \n3....,"2,304,901inClothing,ShoesJewelry(",Turquoise,B0009F7SD8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,B0009NYJF0,Minnetonka Fring Bootie (Infant),[https://images-na.ssl-images-amazon.com/image...,Minnetonka,[Shipping Information:\n \n...,"152,533inClothing,ShoesJewelry(",5 star,B0009NYJF0,NaN,$23.95 - $26.10,"[B0113B91P4, B00LQFGIQ8, B0026MRS0K, B00LUZJL6...","[B0113B91P4, B07HPX7BBT, B004UMIIHG, B00LQFGIQ...","class=""a-normal a-align-center a-spacing-smal...",NaN,NaN,NaN
8,B0009QZAK0,"Rolfs Essentials Black Card, Keys and Coin Wallet",NaN,Rolf's,[Package Dimensions:\n \n5....,NaN,5 star,B0009QZAK0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,B000BCH8AC,Merrell World Passport,NaN,Merrell,NaN,NaN,5 star,B000BCH8AC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
Top25Recommend = top25.merge(data_set,left_on = 'ID', right_on = 'asin')

In [16]:
Top25Recommend[Top25Recommend.columns[:2]]
Top25Recommend = Top25Recommend[Top25Recommend.columns[:2]]

In [17]:
Top25Recommend

,ID,title
0,B0002Z1JNK,Pleaser Women's Electra-2000Z Boot
1,B0002Z1JNK,Pleaser Women's Electra-2000Z Boot
2,B00066G516,Leg Avenue Women's Lace Ruffle Anklet Socks
3,B0007MV6PO,"Henschel Breezer Hat with CoolMax Band Khaki,M..."
4,B0007MV6Q8,"Henschel Breezer Hat with CoolMax Band, Khaki, XL"
5,B00095LIES,Barbie and The Magic of Pegasus: Magical Music...
6,B0009F7SD8,Small Southwestern Native American Handcrafted...
7,B0009NYJF0,Minnetonka Fring Bootie (Infant)
8,B0009QZAK0,"Rolfs Essentials Black Card, Keys and Coin Wallet"
9,B000BCH8AC,Merrell World Passport
